<h1>Namespaces</h1>

În acest tutorial o să trecem peste partea de folosire a 'namespaces' în kubernetes și cele mai bune practivi pentru unde și când să se utilizeze 'namespace' într-un cluster de kubernetes.

<h3> Ce este un 'namespace'? </h3>

În cadrul unui cluster de kubernetes putem să ne organizăm resursele în cadrul acestor 'namespaces'. Putem să avem mai multe 'namespaces' în cadrul unui cluster. Putem să ne imaginăm ca un 'namespace' este un cluster virtual în cadrul unui cluster. În momentul în care se creează un cluster, kubernetes creează 4 'namespaces' în mod default. Dacă rulăm comanda 'kubectl get namespace' putem să vedem aceste 'namespaces' listate.

In [ ]:
kubectl get namespace

<img src='./ScreenShots/Namespaces/01.png'>

<h3>kube-system</h3>

'namespaces'-ul 'kube-system' nu este pentru folosința utilizatorului, prin urmare nu ar trebuie să creem sau să modificăm nimic în cadrul acestui 'namespace'. Componentele pentru care se face deploy în acest 'namespace' sunt procesele de sistem, master managerin processes sau procesele ce țin de kubectl.

<h3>kube-public</h3>

Acest 'namespace' conține datele publice accesibile, un ConfigMap ce conține informații referitoare la cluster, fișier care este accesibil chiar și fără autentificare

<h3>kube-node-lease</h3>

Acest 'namespace' este o adăugare recentă în kubernetes, iar scopul acestui 'namespace' este să rețină informații despre 'viața' pentru nodes. Fiecare node primește propriul obiect care conține informații despre disponibilitatea node-ului

<h3>default</h3>

Acest 'namespace' este cel utilizat pentru a crea componente, asta în cazul în care nu s-a creat alt 'namespace'

Evident se pot creat noi 'namespace' în cadrul unui cluster de kubernetes, iar acest procedeu se realizează utilizând comanda de 'kubectl create namespace'. Comanda trebuie urmată de numele pe care dorim să îl oferim noului 'namespace'.

<img src='./ScreenShots/Namespaces/03.png'>

In [ ]:
kubectl create namespace my-namespace

<img src='./ScreenShots/Namespaces/02.png'>

După ce am utilizat comanda, se poate observa că nou 'namespace' a fost creat. Un alt mod de a crea noi 'namespace' este prin folosirea fișierelor yaml. Acest mod este mai bun deoarece prin acest mod avem și o istorie în cadrul fișierelor de configurație dintr-un anumit repository

<h3>De ce apare nevoia de a crea sau folosi 'namespace'? </h3>

Să ne imagină că folosim doar 'namespace'-ul 'default' oferit de către kubernetes și ne creem toate resursele în cadrul acelui 'namespace'. În situația în care avem o aplicație complexă care conține mai multe componente 'deployment' ce creează mai multe 'pod'-uri, acest 'namespaces' o să fieplin de componente diferite și va fi extrem de dificil de a ne crea o imagine de ansamblu a ceea ce se găsește în acel 'namespace' din kubernetes.

<img src='./ScreenShots/Namespaces/04.png'>

Un mod bun de a rezolva această problemă este de a crea 'namespace' diferite pentru diferitele componente ale aplicației. De exemplu, putem avea un 'namespace' pentru bazele de date (putem să îl denumim 'database'), alt 'namespace' pentru partea de monitorizare ('monitoring'). Un 'namespace' pentru partea de  Elastick Stack se poate crea, unul pentru partea de nginx-ingress. Prin alte cuvinte, gruparea logică a componentelor din cadrul unui cluster.

<img src='./ScreenShots/Namespaces/05.png'>

Conform documentației oficiale kubernetes, nu ar trebui să se utilizeze 'namespace' pentru proiecte mici cu până la 10 utilizatori. Personal, tot timpul este o idee bună să fie grupate componentele din cadrul unui cluster

Un alt exemplu în care trebuie să utilizăm 'namespace' este atunci când avem mai multe echipe. Scenariul în care două echipe folosesc același cluster, iar una dintre echipe face deploy la o aplicație ce poartă denumirea de 'my-app deployment' (numele componentei 'deployment' pe care îl creează). Acest deployment are anumite configurații. Dacă altă echipă face un deploy la o aplicație și din greșeală are același nume ('my-app deployment'), dar o configurație diferită, 'deployment'-ul primei echipe o să fie suprascris. Dacă se va utiliza jenkins pentru acest tip de deployment, cea de-a doua echipă nici nu o să știe că au suprascris 'deployment'-ul primei echipe

<img src='./ScreenShots/Namespaces/06.png'>

Pentru a rezolva acest conflict, putem utiliza 'namespaces' pentru ca fiecare echipă să aibă propirul 'namespace' fără a interveni peste alte echipe.

<img src='./ScreenShots/Namespaces/07.png'>

Un alt exemplu de utilizare de 'namespace' în kubernetes apare atunci când avem un singur cluster, iar în acest cluster dorim să avem partea de 'Staging' a unei aplicații cât și partea de 'Development'. Motivul poate apărea în situația în care aven 'namespace' separat pentru partea de 'Nginx-Ingress Controller' și 'Elastic Stack', putem să utilizăm aceste 'namespace' atât pentru partea de 'Staging' cât și pentru partea de 'Development'. În felul acesta nu trebuie să facem deploy la aceste resurse de 2 ori.

<img src='./ScreenShots/Namespaces/08.png'>

Un alt caz este pentru situația de 'Blue/Green Deployment'. Această situați semnifică că dorim să avem două versiuni diferite de producție ale aceleași aplicații, una care este activă, care este momentan în producție și alta care o să fie următoare versiune din producție. Deși aceste versiuni ale aplicației sunt diferite, acestea pot utiliza aceleași resurse pentru a nu fi nevoie să se facă deploy la acele resurse de două ori. (seamănă cu partea de 'Stagging/Development').

Următorul caz de utilizare a 'namespace' este pentru a limita resursele și accesul la 'namespaces' când se lucrează cu mai multe echipe. Avem din nou un scenariu în care 2 echipe lucrează în același cluster și fiecare echipă are propriul 'namespace'. În acest scenariu putem să le oferim echipelor acces doar în cadrul 'namespace'-ului lor, al echipei din care fac parte. Un alt lucru ce se poate face pentru 'namespace' este să se limiteze resursele pe care le consumă (CPU, RAM, Storage) pentru a oferi echipelor anumite părți din cadru lresurselor disponibile din cluster

<img src='./ScreenShots/Namespaces/09.png'>

<h3>Caracteristicile 'namespaces'-urilor</h3>

Prima caracteristică a acestor 'namespace' este că nu putem accesa cele mai multe resurse din alt 'namespace'. Dacă avem un ConfigMap în cadrul unui 'namespace' (Project A) prin care se face referire la un service de bază de date din alt namespace (database), nu putem utiliza acel ConfigMap în cadrul altui 'namespace' (Project B). Va trebui să creem același ConfigMap care fare referință la același service de bază de date. Fiecare 'namespace' trebuie să aibă propriun ConfigMap. La fel se procedează și pentru Secret, fiecare 'namespace' trebuie să aibă propria componentă Secret. O resursă care se poate accesa în mai multe 'namespace' diferite este cea Service. În cadrul unui ConfigMap, url-ul unei baze de date, pe lângă numele service-ului o să conțină și numele 'namespace'-ului. 'database_url: mysql-service.database' 

'.database', trecut după numele de Service la care dorim să face referire reprezintă numele 'namespace'-ului de unde dorim să facem acea referire.

<img src='./ScreenShots/Namespaces/10.png'>

O altă caracteristică ce ține de 'namespace' este faptul că anumite componente din cadrul kubernetes care nu sunt prezente în niciun 'namespace', acestea sunt prezente la nivel global în cadrul cluster-ului și nu avem posibilitatea de a le izola sau de a le pune într-un anumit 'namespace'. Un exemplu pentru aceste componente este partea de 'Volumes' sau 'Nodes'. În situația în care se creează un 'Volume' acesta o să fie accesibil din oricare 'namespace'. Aceste componente care nu depind de un 'namespace' se pot lista utilizând comanda 'kubectl api-resources --namespace=false'

In [ ]:
kubectl api-resources --namespace=false

În continuare o să creem anumite componente în cadrul unui 'namespace'. În exemplu de proiect demo am creat creat componente utilizând fișiere yaml. Nicăieri în cadrul acelui fișier nu am definit partea de 'namespace'. Dacă nu se se specifică un 'namespace' pentru o componentă, aceasta o să fie trecută în 'namespace'-ul 'default' O să utilizâm un fișier yaml pentru a crea componente în diferite 'namespace'-uri

<img src='./ScreenShots/Namespaces/11.png'>

Fișierul de mai sus este fișieurl cu care o să lucrăm. Dacă utilizăm comanda 'kubectl apply -f mysql-configmap.yaml', componenta ConfigMap va fi creată. Dacă rulăm comanda 'kubectl get configmap' o să ni se afișeze componenta ConfigMap ce tocmai am creat-o. De notat faptul că în cadrul acestei comenzi nu am specificat un 'namespace' de unde să ne ia componenta respectică, în mod default, comanda la rulare ia componentele din 'namespace'-ul 'default'. Se poate specifica ce 'namespace' să se ia în considerare în momentul în care se utilizează o comandă prin utilizare flag-ului '-n'. La acest flag trebuie specificat numele 'namespace'-ului.

In [ ]:
kubectl get configmap

In [ ]:
kubectl get configmap -n default

Cele două comenzi de mai sus sunt identice. Comanda de 'aplly' are și ea un flag pentru partea de 'namespace', iar acesta poartă denumirea de '--namespace'. Acestui flag trebuie să îi specificăm numele 'namespaces'-ului unde dorim să creem componenta respectivă. O să creem aceeași componentă ConfigMap, dar de data aceasta în cadrul 'namespace'-ului 'my-namespace' ce l-am creat la începutul tutorialului.

In [ ]:
kubectl apply -f mysql-configmap.yaml --namespace=my-namespace

<img src='./ScreenShots/Namespaces/12.png'>

Acesta este un mod de a crea componente în cadrul unui 'namespace'. Un alt mod este să se realizeze etapa aceasta direct în cadrul fișierului yaml. Pentru a face acest lucru, în secțiunea 'metadata' se va trece un set de valori key-value, unde partea de key va fi 'namespace', iar partea de value o să fie numele 'namespace'-ului unde dorim să creem această componentă.

<img src='./ScreenShots/Namespaces/13.png'>

O astfel de componentă deja există în cadrul acestui 'namespace', a fost creată anterior. Pentru a putea crea din nou componenta resepctivă trebuie să ștergem componenta inițială. Pentru aceasta o să utilizăm comanda 'kubectl delete configmap'. Acestei comenzi i se va oferi ca și argument numele componentei 'ConfigMap' ce dorim să o ștergem (mysql-configmap). Deoarece comanda 'kubectl' caută defaul în 'namespace'-ul default, trebuie să îi specificăm și din ce 'namespace' să șteargă această componentă. Pentru aceasta, comenzii o să ți adăugăm și argumentul '-n=my-namespace'

In [ ]:
kubectl delete configmap mysql-configmap -n=my-namespace

<img src='./ScreenShots/Namespaces/14.png'>

După ce am realizat acest pas, în acest moment putem rula comanda 'kubectl apply -f mysql-configmap.yaml'. Din moment ce am adăugat atributul 'namespace: my-namespace' în 'metadata', kubernetes automat o să creeze componenta respectivă în cadrul 'namespace'-ului denumit 'my-namespace'.

<img src='./ScreenShots/Namespaces/15.png'>

Comanda de mai sus a creat cu succes componenta 'configmap' în cadrul 'namespace'-ului 'my-namespace'. Se recomandă să se folosească atributul de 'namespace' în cadrul fișierului de configurație (yaml) deoarece este documentat mai bine așa, doar uitându-ne în fișier ne dăm seama unde anume este creată componenta respectivă, iar dacă se utilizează parte de deployment automatizată, atunci este mult mai convenabil să fie trecut 'namespace' în fișierul yaml.